In [1]:
import psycopg2
import numpy as np
import pandas as pd

In [2]:
# Connect to database
con = psycopg2.connect("dbname='SNZLab' user='postgres' host='localhost' password='PlantDevSchnLab112022'")

# Open a cursor to perform database operations
cur = con.cursor()

# Execute a command
cur.execute("SELECT * FROM cell_attributes")

In [3]:
# Obtain data as python opject

# Obtain first row of data table
# first_col = cur.fetchone()

# Obtain all rows
cell_attributes = cur.fetchall()

# Obtain column names
colnames = [desc[0] for desc in cur.description]

In [4]:
# Create pandas DataFrame from list of tuples
df = pd.DataFrame(cell_attributes)

# Add column names to respective columns
df.columns = colnames

In [41]:
df.head()

,cell_id,parent_label,genotype,sample_id,stage,cell_length_x,cell_length_y,cell_length_z,cell_wall_area,number_of_neighbors,outside_wall_area,outside_wall_area_ratio,volume,parent_name,tissue_label
0,1835,7,ChiOX,1793,3-IV,6.00088,5.29523,6.09765,148.201,3,108.799,0.734131,142.288,fu,fu
1,1838,7,ChiOX,1793,3-IV,5.57317,4.97323,8.43972,147.736,7,105.308,0.712816,131.511,fu,fu
2,1842,7,ChiOX,1793,3-IV,7.01422,5.10685,9.65658,205.962,9,101.471,0.492667,228.485,fu,fu
3,1846,7,ChiOX,1793,3-IV,6.57697,4.84003,10.2757,221.655,13,114.535,0.516727,245.79,fu,fu
4,1847,7,ChiOX,1793,3-IV,5.87977,5.71539,6.19016,135.858,7,69.4824,0.511433,132.188,fu,fu


In [ ]:
# Add parent and tissue label according to variable "parent_id"
df['parent_name']=df['parent_label'].apply(lambda x: 'oi2' if x==1 else ('oi1' if x==2 else ('ii2' if x==3 else ('ii1' if x==4 else ('nu' if x==5 else ('PCh' if x==6 else ('fu' if x==7 else ('es' if x==8 else ('pl' if x==9 else 'ACh')))))))))
df['tissue_label'] = df['parent_label'].apply(lambda x: 'oi' if x==1 or x==2 else ('ii' if x==3 or x==4 else ('ch' if x==6 or x==10 else ('nu' if x==5 else ('fu' if x==7 else ('es' if x==8 else 'pl'))))))

In [42]:
# Count unique values in variable "sample_id"
count_sample = df['sample_id'].value_counts()
print(count_sample)

1793    1803
Name: sample_id, dtype: int64


In [43]:
# Group by variables "sample_id", "stage" and sum up the volume
volume_sample = df.groupby(['sample_id', 'stage'])['volume'].sum().reset_index()
print(volume_sample)

   sample_id stage          volume
0       1793  3-IV  549413.2776469


In [44]:
# Group by variables "sample_id", "stage" and sum up the volume
tissue_volume_sample = df.groupby(['stage', 'sample_id', 'tissue_label'])['volume'].sum().reset_index()
print(tissue_volume_sample)

  stage  sample_id tissue_label         volume
0  3-IV       1793           ch   89611.349193
1  3-IV       1793           es         5820.9
2  3-IV       1793           fu   84846.750602
3  3-IV       1793           ii     82750.5200
4  3-IV       1793           nu    13064.33455
5  3-IV       1793           oi  265555.296438
6  3-IV       1793           pl   7764.1268639


In [45]:
# Group by variables "sample_id", "stage" and count unique values in variable "parent"
count_parent_by_sample = df.groupby(['sample_id', 'stage'])['parent_label'].value_counts().sort_index(ascending=True)
print(count_parent_by_sample)

sample_id  stage  parent_label
1793       3-IV   0                29
                  1               238
                  2               188
                  3               140
                  4               126
                  5                92
                  6               261
                  7               528
                  8                 1
                  10              171
                  14               29
Name: parent_label, dtype: int64


In [32]:
# Group by variables "stage" and count unique values in variable "sample_id"
count_sample_by_stage = df.groupby(['stage'])['sample_id'].value_counts()
print(count_sample_by_stage)

stage  sample_id
3-IV   1793         1803
Name: sample_id, dtype: int64
